## Notebook setup

In [ ]:
import warnings
import numpy as np
import datetime

from IPython.display import HTML

import openbb_terminal.config_terminal as cfg
from openbb_terminal import OpenBBFigure, theme
from openbb_terminal.helper_classes import TerminalStyle
from openbb_terminal.core.plots.plotly_ta.ta_class import PlotlyTA

warnings.filterwarnings("ignore")

In [ ]:
# Suppressing sdk logs
from openbb_terminal.core.session.current_system import set_system_variable

set_system_variable("LOGGING_SUPPRESS", True)

# Import the OpenBB SDK
from openbb_terminal.sdk import openbb, widgets

In [ ]:
from openbb_terminal.core.session.current_user import get_current_user

user = get_current_user()

cfg.theme = TerminalStyle(
    user.preferences.MPL_STYLE,
    user.preferences.PMF_STYLE,
    user.preferences.RICH_STYLE,
)
theme.apply_style("light")

In [ ]:
stylesheet = widgets.html_report_stylesheet()

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
symbol = "GLD"
report_name = "Equity Report for VTI"

In [ ]:
ticker = symbol

In [ ]:
if "." in ticker:
    import sys

    sys.exit(0)

In [ ]:
ticker_data = openbb.stocks.load(
    symbol=ticker, start_date=datetime.datetime.now() - datetime.timedelta(days=4 * 30)
)
ticker_data = openbb.stocks.process_candle(data=ticker_data)

author = "OpenBB"
report_title = f"ETF RESEARCH REPORT ON {ticker.upper()}"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

In [ ]:
# info = openbb.etf.summary(name=ticker)
# info

## Data

In [ ]:
data = openbb.stocks.load(ticker)

candle_chart = openbb.etf.candle(
    symbol=ticker,
    data=data,
    external_axes=True,
)

if candle_chart:
    candle_chart = candle_chart.to_html()

In [ ]:
import pandas as pd

sectors = openbb.etf.weights(ticker)
sector_weights_formatted = {}
for sector_weight in sectors:
    sector_weights_formatted[sector_weight["sector"]] = (
        float(sector_weight["weightPercentage"].strip("%")) / 100
    )
sector_weights_formatted = dict(sorted(sector_weights_formatted.items()))
weights_df = pd.DataFrame.from_dict(sector_weights_formatted, orient="index")

In [ ]:
weights_chart = ""

if "Percentage" in weights_df.columns:
    weights_df.columns = ["Percentage"]

if (
    "Percentage" in weights_df.columns
    and len(weights_df[weights_df["Percentage"] > 10].index) > 1
):
    colors = theme.get_colors()
    fig = OpenBBFigure.create_subplots(
        1,
        3,
        specs=[[{"type": "domain"}, {"type": "pie", "colspan": 2}, None]],
        row_heights=[1],
        column_widths=[0.1, 0.8, 0.1],
    )

    fig.add_pie(
        labels=weights_df[weights_df["Percentage"] > 10].index,
        values=weights_df[weights_df["Percentage"] > 10]["Percentage"],
        textinfo="label+percent",
        hoverinfo="label+percent",
        automargin=True,
        rotation=45,
        row=1,
        col=2,
    )
    fig.update_traces(
        textposition="outside",
        textfont_size=15,
        marker=dict(
            colors=colors,
            line=dict(color="#F5EFF3", width=0.8),
        ),
    )

    fig.update_layout(
        margin=dict(t=40, b=20),
        title=dict(
            text=f"Sector holdings of {ticker}",
            y=0.98,
            x=0.5,
            xanchor="center",
            yanchor="top",
        ),
        colorway=colors,
        showlegend=False,
    )

    weights_chart = fig.to_html()

In [ ]:
holdings = openbb.etf.holdings(ticker)
holdings

In [ ]:
indicators = dict(
    ema=dict(length=[50]),
    sma=dict(length=[50]),
    wma=dict(length=[50]),
)

ma_chart = PlotlyTA.plot(ticker_data, indicators, symbol, candles=False, volume=False)

if ma_chart:
    ma_chart = ma_chart.show(external=True)
    ma_chart = ma_chart.to_html()

In [ ]:
macd_chart = openbb.ta.macd_chart(
    ticker_data["Close"], symbol=symbol, external_axes=True
)

if macd_chart:
    macd_chart = macd_chart.to_html()

In [ ]:
cci_chart = openbb.ta.cci_chart(ticker_data, symbol=symbol, external_axes=True)

if cci_chart:
    cci_chart = cci_chart.to_html()

In [ ]:
stoch_chart = openbb.ta.stoch_chart(ticker_data, symbol=symbol, external_axes=True)

if stoch_chart:
    stoch_chart = stoch_chart.to_html()

In [ ]:
adx_chart = openbb.ta.adx_chart(ticker_data, symbol=symbol, external_axes=True)

if adx_chart:
    adx_chart = adx_chart.to_html()

In [ ]:
bbands_chart = openbb.ta.bbands_chart(ticker_data, symbol=symbol, external_axes=True)

if bbands_chart:
    bbands_chart = bbands_chart.to_html()

In [ ]:
ad_chart = openbb.ta.ad_chart(ticker_data, symbol=symbol, external_axes=True)

if ad_chart:
    ad_chart = ad_chart.to_html()

In [ ]:
ta_rsi = openbb.ta.rsi_chart(ticker_data["Close"], external_axes=True)

if ta_rsi:
    ta_rsi = ta_rsi.to_html()

In [ ]:
df = openbb.ta.rsi(ticker_data["Close"])
rsi_value = round(df.values[-1][0], 2)
rsi_value

## Render the report template to a file

In [ ]:
body = ""


body += widgets.header(
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>ETF RESEARCH REPORT:</b> {ticker.upper()}",
    plotly_js=True,
)

body += widgets.tablinks(
    [
        "SUMMARY",
        "Technical Analysis",
    ]
)

htmlcode = widgets.h(3, "Summary")
# htmlcode += widgets.p(info)
htmlcode += widgets.row([candle_chart])

htmlcode += widgets.p("The ETF has the following weights distribution:")

htmlcode += widgets.row([weights_df.to_html()])
htmlcode += widgets.row([weights_chart])

htmlcode += widgets.p("And has funds in the following assets:")
htmlcode += widgets.row([holdings.to_html()])

body += widgets.add_tab("SUMMARY", htmlcode)

htmlcode = widgets.row([widgets.h(3, f"Moving Averages for {symbol}") + ma_chart])
htmlcode += widgets.row([macd_chart or ""])
htmlcode += widgets.row([ta_rsi or ""])
htmlcode += widgets.row([stoch_chart or ""])
htmlcode += widgets.row([cci_chart or ""])
htmlcode += widgets.row([ad_chart or ""])
htmlcode += widgets.row([bbands_chart or ""])
htmlcode += widgets.row([adx_chart or ""])
body += widgets.add_tab("Technical Analysis", htmlcode)


body += widgets.tab_clickable_and_save_evt()

report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)


# to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)